Это решение берёт ориентировочно 92.8 на Public LeaderBoard (5 место на текущий момент)

Что было опробовано и не зажгло:
- Стакинг моделей. Был испробован стакинг сильной модели (Random Forest) и логистической регресси + KNN. Мета классифиикатором был выбран катбуст с категориальными фичами (месяц, день недели, субъект РФ, область РФ).
- Были опробованы счётчики событий в радиусе с течением времени - к сожалению, очень сильно дропнули валидацию...
- Были опробованы счётчики событий в субъекте, области, номере (поделил всю РФ на сетку) ячейки в прошлом году в том же месяце.
- Нейронная сеть (полагаю, что это из-за разнородности данных)

Что в свою очеред зажгло:
- Категориальные фичи в катбусте (месяц, день недели, субъект РФ и область)
- Минимальное расстояние до пожара определёного типа
- 20-е по удалённости значение расстояния до каждого типа пожаров (таким образом показывается компактность каждого типа пожаров).
- Также хорошей идеей стало взятие ncep данных на разных высотах (средняя высота над уровнем моря в России 150 метров, тогда как в бейзлайне были использованы только данные на высоте 1000 метров) + хорошей идеей стало взятие vwind данных из ncep.
- Также зажгли счётчики по количеству событий в субъекте, области и немере ячейки за последний месяц (к сожалению, в итоговой модели их использовать нельзя, так как учился я на анных до апреля, а на тесте будет декабрь без возможности дообучения :( ) 
    

Заимпортим все необходимые библиотеки.

In [492]:
import pandas
from datetime import date
import xarray
import requests
import datetime
import numpy as np
import gzip
from tqdm import tqdm as tqdm
from tqdm import tqdm_notebook
import os
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from math import cos, asin, sqrt
import pandas as pd
import warnings 
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import datetime
from sklearn.ensemble import GradientBoostingClassifier
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.mixture import BayesianGaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from multiprocessing import Process
import multiprocessing
import mlens
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
warnings.simplefilter('ignore')

height_surf = xarray.open_dataset("./data/hgt.sfc.nc")
land = xarray.open_dataset("./data/land.nc")
nceps = [[], [], [], [], [], [], [], []]
nceps_sfc = [[], [], [], [], [], [], [], []]
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

for index, year in enumerate(years):
    for var in ('hgt', 'srfpt', 'trpp', 'trpt', 'air', 'uwnd', 'vwnd', 'rhum'):
        dataset_filename = './data/{}.{}.nc'.format(var, year)
        nceps[index].append(xarray.open_dataset(dataset_filename))
    nceps[index] = xarray.merge(nceps[index])
    
for index, year in enumerate(years):
    for var in ('air.sfc', 'rhum.sfc', 'vwnd.sfc'):
        dataset_filename = './data/{}.{}.nc'.format(var, year)
        nceps_sfc[index].append(xarray.open_dataset(dataset_filename))
    nceps_sfc[index] = xarray.merge(nceps_sfc[index])

Как и говорилось выше, давайте смотреть данные ncep на разных высотах. Так как вычисление этих фичей занимает очень большое время - внутри предлагается это распараллелеить. 

In [2]:
df_train = pandas.read_csv('./wildfires-data-train/wildfires_train.csv')
    
def extract_features(row, index):
    date = row['date']
    dt = datetime.date(int(date[:4]), int(date[5:7]), int(date[8:10]))
    levels = [100, 150, 200, 500, 1000]
    answer = {
        'fire_id': row['fire_id'],
        'month': dt.month,
        'day': dt.day,
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'longitude': row['longitude'],
        'latitude': row['latitude'],
        'date': row['date'],
        'weekday': dt.weekday()}
    
    for level in levels:
        point = nceps[index].sel(
            lon=row['longitude'],
            lat=row['latitude'],
            level=level,
            method='nearest',
        )
        
        p3w = point.rolling(time=21).mean()
        p1w = point.rolling(time=7).mean()
        p2w = point.rolling(time=14).mean()

        v = point.sel(time=date)
        v1w = p1w.sel(time=date)
        v2w = p2w.sel(time=date)
        v3w = p3w.sel(time=date)
        
        answer['temperature ' + str(level)] = v.air.values.item(0)
        answer['humidity ' + str(level)] = v.rhum.values.item(0)
        answer['uwind ' + str(level)] = v.uwnd.values.item(0)
        answer['temperature_1w_mean ' + str(level)] = v1w.air.values.item(0)
        answer['temperature_2w_mean ' + str(level)] = v2w.air.values.item(0)
        answer['temperature_3w_mean ' + str(level)] = v3w.air.values.item(0)
        answer['humidity_1w_mean ' + str(level)] = v1w.rhum.values.item(0)
        answer['humidity_2w_mean ' + str(level)] = v2w.rhum.values.item(0)
        answer['humidity_3w_mean ' + str(level)] = v3w.rhum.values.item(0)
        answer['height_1w_mean' + str(level)] = v1w.hgt.values.item(0)
        answer['height_2w_mean' + str(level)] = v2w.hgt.values.item(0)
        answer['height_3w_mean' + str(level)] = v3w.hgt.values.item(0)
        answer['srfpt_1w_mean' + str(level)] = v1w.srfpt.values.item(0)
        answer['srfpt_2w_mean' + str(level)] = v2w.srfpt.values.item(0)
        answer['srfpt_3w_mean' + str(level)] = v3w.srfpt.values.item(0)
        answer['trpp_1w_mean' + str(level)] = v1w.trpp.values.item(0)
        answer['trpp_2w_mean' + str(level)] = v2w.trpp.values.item(0)
        answer['trpp_3w_mean' + str(level)] = v3w.trpp.values.item(0)
        answer['trpt_1w_mean' + str(level)] = v1w.trpt.values.item(0)
        answer['trpt_2w_mean' + str(level)] = v2w.trpt.values.item(0)
        answer['trpt_3w_mean' + str(level)] = v3w.trpt.values.item(0)
        answer['vwnd_1w_mean' + str(level)] = v1w.vwnd.values.item(0)
        answer['vwnd_2w_mean' + str(level)] = v2w.vwnd.values.item(0)
        answer['vwnd_3w_mean' + str(level)] = v3w.vwnd.values.item(0)
        answer['uwnd_1w_mean' + str(level)] = v1w.uwnd.values.item(0)
        answer['uwnd_2w_mean' + str(level)] = v2w.uwnd.values.item(0)
        answer['uwnd_3w_mean' + str(level)] = v3w.uwnd.values.item(0)
        
    point = nceps_sfc[index].sel(
        lon=row['longitude'],
        lat=row['latitude'],
        method='nearest',
    )

    p3w = point.rolling(time=21).mean()
    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()

    v = point.sel(time=date)
    v1w = p1w.sel(time=date)
    v2w = p2w.sel(time=date)
    v3w = p3w.sel(time=date)

    answer['temperature_sfc'] = v.air.values.item(0)
    answer['humidity_sfc'] = v.rhum.values.item(0)
    answer['temperature_1w_mean_sfc'] = v1w.air.values.item(0)
    answer['temperature_2w_mean_sfc'] = v2w.air.values.item(0)
    answer['temperature_3w_mean_sfc'] = v3w.air.values.item(0)
    answer['vwnd_1w_mean_sfc'] = v1w.vwnd.values.item(0)
    answer['vwnd_2w_mean_sfc'] = v2w.vwnd.values.item(0)
    answer['vwnd_3w_mean_sfc'] = v3w.vwnd.values.item(0)
    answer['humidity_1w_mean_sfc'] = v1w.rhum.values.item(0)
    answer['humidity_2w_mean_sfc'] = v2w.rhum.values.item(0)
    answer['humidity_3w_mean_sfc'] = v3w.rhum.values.item(0)
        
    return answer

def get_interval_features(query, index, features_list):
    df_subsample = df_train.query(query)
    df_features = []
    for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
        features = extract_features(row, index)
        df_features.append(features)
    df_features = pandas.DataFrame(df_features).sort_values('date')
    df_features.set_index('fire_id', inplace=True)
    features_list[index] = df_features

def extract_all_basic_features(queries):
    manager = multiprocessing.Manager()
    return_dict = manager.dict()
    pool = []
    
    for index, query in enumerate(queries):
        pool.append(Process(target=get_interval_features, args=(query, index, return_dict)))
        pool[index].start()

    for thread in pool:
        thread.join()
    
    return return_dict

# query1213 = '(date > "2012") & (date < "2013")' #распределение сильно отличается
# query1314 = '(date > "2013") & (date < "2014")'
# query1415 = '(date > "2014") & (date < "2015")'
# query1516 = '(date > "2015") & (date < "2016")'
# query1617 = '(date > "2016") & (date < "2017")'
# query1718 = '(date > "2017") & (date < "2018")'
# query1819 = '(date > "2018") & (date < "2019")'
# query1920 = '(date > "2019") & (date < "2020")'

# queries = [query1213, query1314, query1415, query1516, query1617, query1718, query1819, query1920]
# queries_titles = ['2012-2013', '2013-2014', '2014-2015', '2015-2016',
#                   '2016-2017', '2017-2018', '2018-2019', '2019-2020']

# years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']

# features_list = extract_all_basic_features(queries).values()

Очень важным шагом был сбор данных о крупнейших российских городах (областях и субъектах, в которых они находятся).
Данных файл будет приложен в репозитории. Также в в ноутбуке EDA будет код, для качивания фотографий со спутника для наглядного 
представления территорий, на которых был пожар, на нх будет очень хорошо видно, какие территории покрыты чисто лесом, а какие
покрыты полями, что сильно поможет различать контролируемый пал / неконтролируемый пал от лесного и природного подаров.
Для классификации типа местности на изображении была использована нейронная сеть ResNet50 (ноутбук появится позже).
Далее всё это было переведено в файлы field_cords.csv и forest_coords.csv. Стоит заметить что изображения экстрактились
с шагом примерно 0.2 градуса по широте и долготе. Это очень помогло модели различать типы пожаров. В коде ниже мы просто считываем
уже подготовленные данные из файлов, которые можно будет найти в данном репозитории.

In [3]:
cities_info = pd.read_json('./data/russian-cities.json')
cities_info['latitude'] = cities_info.apply(lambda x: x.coords['lat'], axis=1)
cities_info['longitude'] = cities_info.apply(lambda x: x.coords['lon'], axis=1)
cities_info = cities_info.drop(columns=['coords'])
cities_info['subject'] = cities_info['subject'].astype('category').cat.codes
cities_info['district'] = cities_info['district'].astype('category').cat.codes

forest_info = pd.read_csv('forest_coords.csv')
field_info = pd.read_csv('field_coords.csv')
field_info['field'] = field_info['field'].apply(lambda x: 2)
merged_land_df = pd.concat([forest_info.rename({'forest': 'land_indx'}, axis=1),
                            field_info.rename({'field': 'land_indx'}, axis=1)]).reset_index()
forests_df = pd.read_csv('nature_forests.csv')

Самые основные фнкции для feature generation на основе KNN.

In [351]:
def get_count_subj_month_only_prev(df_train, df_test, field_type='subject'):
    df_train_dt = df_train.copy()
    df_test_dt = df_test.copy()
    merged_df = pd.concat([df_train, df_test]).reset_index(drop=True)
    
    date_area_count = merged_df.groupby(['date', field_type]).count().reset_index()
    df_area_counters = pd.DataFrame(columns=['events_amount_passed_in_' + str(field_type)])
    for index, row in tqdm(date_area_count.iterrows()):
        year = int(row.date[:4])
        month = int(row.date[5:7])
        day = row.date[8:]
        if month == 1: 
            prev_month = "12"
            prev_year = str(year - 1)
        else:
            prev_month = str(month - 1)
            if len(prev_month) == 1:
                prev_month = "0" + prev_month
            prev_year = str(year)
        amount_events = date_area_count[(date_area_count[field_type] == row[field_type]) &
                      (date_area_count.date >= str(prev_year) + "-" + str(prev_month) + str(day)) &
                      (date_area_count.date <= row.date)].month.sum()
        
        df_area_counters = df_area_counters.append(pd.DataFrame(np.array(amount_events).reshape(-1, 1), 
                                                                columns=['events_amount_passed_last_month_in_' + str(field_type)]))
        
    df_area_counters = df_area_counters.reset_index(drop=True)
    
    df_area_counters = pd.concat([df_area_counters, date_area_count], axis=1)[['date', field_type, 
                                                                          'events_amount_passed_last_month_in_' + str(field_type)]]
    df_train_dt = df_train_dt.merge(df_area_counters, how='left', on=['date', field_type])
    df_test_dt = df_test_dt.merge(df_area_counters, how='left', on=['date', field_type])
    
    return df_train_dt, df_test_dt

def get_count_subj_month(df_train, df_test, field_type='subject'):
    df_train_dt = df_train.copy()
    df_test_dt = df_test.copy()
    merged_df = pd.concat([df_train, df_test]).reset_index(drop=True)
    
    date_area_count = merged_df.groupby(['date', field_type]).count().reset_index()
    df_area_counters = pd.DataFrame(columns=['events_amount_passed_in_' + str(field_type)])
    area_count = {}
    for index, row in date_area_count.iterrows():
        if row[field_type] in area_count.keys():
            area_count[row[field_type]] += row['month']
        else:
            area_count[row[field_type]] = row['month']
        df_area_counters = df_area_counters.append(pd.DataFrame([area_count[row[field_type]]], 
                                                                columns=['events_amount_passed_in_' + str(field_type)]))
        
    df_area_counters = df_area_counters.reset_index(drop=True)
    
    df_area_counters = pd.concat([df_area_counters, date_area_count], axis=1)[['date', field_type, 
                                                                          'events_amount_passed_in_' + str(field_type)]]
    df_train_dt = df_train_dt.merge(df_area_counters, how='left', on=['date', field_type])
    df_test_dt = df_test_dt.merge(df_area_counters, how='left', on=['date', field_type])
    
    return df_train_dt, df_test_dt

def get_radius_counters(df_train, df_test, fire_types, radiuses=[1.5]):
    all_answers_train, all_answers_test = [], []
    all_min_dist_train, all_min_dist_test = np.full((df_train.shape[0], len(fire_types)), 1000.0).astype(float),  np.full((df_test.shape[0], len(fire_types)), 1000).astype(float)
    all_max_dist_train, all_max_dist_test = np.full((df_train.shape[0], len(fire_types)), 1000.0).astype(float),  np.full((df_test.shape[0], len(fire_types)), 1000).astype(float)
    
    for radius in tqdm(radiuses):
        answers_train = np.full((df_train.shape[0], len(fire_types)), 1000.0).astype(float)
        answers_test = np.full((df_test.shape[0], len(fire_types)), 1000.0).astype(float)
        answers_min_dist_train = np.full((df_train.shape[0], len(fire_types)), 1000.0).astype(float)
        answers_min_dist_test = np.full((df_test.shape[0], len(fire_types)), 1000.0).astype(float)
        for fire_index, fire_type in enumerate(fire_types):
            neigh = NearestNeighbors(2, radius=radius)
            neigh.fit(df_train[df_train.fire_type_name == fire_type][['longitude', 'latitude']])
            
            dist_train, indices_train = neigh.radius_neighbors(df_train[['longitude', 'latitude']])
            dist_test, indices_test = neigh.radius_neighbors(df_test[['longitude', 'latitude']])
            dist_min_train, _ = neigh.kneighbors(df_train[['longitude', 'latitude']])
            dist_min_test, _ = neigh.kneighbors(df_test[['longitude', 'latitude']])
            
            for indx, val in enumerate(dist_train):
                ans = len(val)
                if val.shape[0] > 0 and val.min() < 1e-9:
                    ans -= 1
                if dist_min_train[indx].min() < 1e-9:
                    answers_min_dist_train[indx][fire_index] = dist_min_train[indx].max()
                else:
                    answers_min_dist_train[indx][fire_index] = dist_min_train[indx].min()
                answers_train[indx][fire_index] = ans
            for indx, val in enumerate(dist_test):
                ans = len(val)
                if val.shape[0] > 0 and val.min() < 1e-9:
                    ans -= 1
                if dist_min_test[indx].min() < 1e-9:
                    answers_min_dist_test[indx][fire_index] = dist_min_test[indx].max()
                else:
                    answers_min_dist_test[indx][fire_index] = dist_min_test[indx].min()
                answers_test[indx][fire_index] = ans
                
        all_answers_train.append(answers_train.astype(int))
        all_answers_test.append(answers_test.astype(int))
        all_min_dist_train = answers_min_dist_train
        all_min_dist_test = answers_min_dist_test
    
    for fire_index, fire_type in tqdm(enumerate(fire_types)):
        neigh = NearestNeighbors(20)
        neigh.fit(df_train[df_train.fire_type_name == fire_type][['longitude', 'latitude']])

        dist_min_train, _ = neigh.kneighbors(df_train[['longitude', 'latitude']])
        dist_min_test, _ = neigh.kneighbors(df_test[['longitude', 'latitude']])

        for indx, val in enumerate(dist_min_train):
            all_max_dist_train[indx][fire_index] = dist_min_train[indx].max()
        for indx, val in enumerate(dist_min_test):
            all_max_dist_test[indx][fire_index] = dist_min_test[indx].max()
            
    return all_answers_train, all_answers_test, all_min_dist_train, all_min_dist_test, all_max_dist_train, all_max_dist_test

def get_radius_cities_amount(df_train, df_test, nearest_cities, radiuses=[1.5]):
    all_answers_train, all_answers_test = [], []
    all_districts_train, all_districts_test = [], []
    all_subj_train, all_subj_test = [], []
    min_dist_train, min_dist_test = np.full((df_train.shape[0], 1), 1000.0).astype(float), np.full((df_test.shape[0], 1), 1000.0).astype(float)
    
    for radius in tqdm(radiuses):
        answers_train = np.full((df_train.shape[0], 1), 1000.0).astype(float)
        answers_test = np.full((df_test.shape[0], 1), 1000.0).astype(float)
        
        neigh = NearestNeighbors(10, radius=radius)
        neigh.fit(nearest_cities[['longitude', 'latitude']])
        dist_train, indices_train = neigh.radius_neighbors(df_train[['longitude', 'latitude']])
        dist_test, indices_test = neigh.radius_neighbors(df_test[['longitude', 'latitude']])
        
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(nearest_cities[['longitude', 'latitude']], nearest_cities['subject'])
        subject_train = knn.predict(df_train[['longitude', 'latitude']])
        subject_test = knn.predict(df_test[['longitude', 'latitude']])
        
        knn.fit(nearest_cities[['longitude', 'latitude']], nearest_cities['district'])
        district_train = knn.predict(df_train[['longitude', 'latitude']])
        district_test = knn.predict(df_test[['longitude', 'latitude']])
        
        neigh_inf = NearestNeighbors(10, radius=200)
        neigh_inf.fit(nearest_cities[['longitude', 'latitude']])
        min_dist_train, min_ind_train = neigh_inf.radius_neighbors(df_train[['longitude', 'latitude']])
        min_dist_test, min_ind_test = neigh_inf.radius_neighbors(df_test[['longitude', 'latitude']])
                
        for indx, val in enumerate(indices_train):
            answers_train[indx][0] = len(val)
            min_dist_train[indx] = min_dist_train[indx].min()
        for indx, val in enumerate(indices_test):
            answers_test[indx][0] = len(val)
            min_dist_test[indx] = min_dist_test[indx].min()
        
        all_answers_train.append(answers_train.astype(int))
        all_answers_test.append(answers_test.astype(int))
        all_districts_train = district_train
        all_districts_test = district_test
        all_subj_train = subject_train
        all_subj_test = subject_test
    
    return all_answers_train, all_answers_test, all_districts_train, all_districts_test, all_subj_train, all_subj_test, min_dist_train, min_dist_test

def get_radius_land_types_amount(df_train, df_test, land_type_df, radiuses=[1.5]):
    all_am_train, all_am_test = [], []
    min_dist_train, min_dist_test = np.full((df_train.shape[0], 1), 1000.0).astype(float), np.full((df_test.shape[0], 1), 1000.0).astype(float)
    
    neigh_inf = NearestNeighbors(1)
    neigh_inf.fit(land_type_df[['longitude', 'latitude']])
    min_dist_train, min_indices_train = neigh_inf.kneighbors(df_train[['longitude', 'latitude']])
    min_dist_test, min_indices_test = neigh_inf.kneighbors(df_test[['longitude', 'latitude']])
    
    for radius in tqdm(radiuses):
        am_train, am_test = np.zeros((df_train.shape[0], 1)), np.zeros((df_test.shape[0], 1))
        
        neigh = NearestNeighbors(10, radius=radius)
        neigh.fit(land_type_df[['longitude', 'latitude']])
        dist_train, indices_train = neigh.radius_neighbors(df_train[['longitude', 'latitude']])
        dist_test, indices_test = neigh.radius_neighbors(df_test[['longitude', 'latitude']])
                
        for indx, val in enumerate(dist_train):
            am_train[indx][0] = len(val)
            min_dist_train[indx] = min_dist_train[indx].min()
        for indx, val in enumerate(dist_test):
            am_test[indx][0] = len(val)
            min_dist_test[indx] = min_dist_test[indx].min()
        
        all_am_train.append(am_train)
        all_am_test.append(am_test)
    
    return min_dist_train, min_dist_test, all_am_train, all_am_test

def get_land_class_knn(df_train, df_test, land_type_df):    
    answers_train = np.zeros((df_train.shape[0], 1))
    answers_test = np.zeros((df_test.shape[0], 1))
    am_train, am_test = np.zeros((df_train.shape[0], 1)), np.zeros((df_test.shape[0], 1))

    neigh = KNeighborsClassifier(n_neighbors=5)
    neigh.fit(land_type_df[['longitude', 'latitude']], land_type_df['land_indx'])
    land_class_train = neigh.predict(df_train[['longitude', 'latitude']])
    land_class_test = neigh.predict(df_test[['longitude', 'latitude']])
    
    return land_class_train, land_class_test

def get_forests_min_dist(df_train, df_test, forests_df):        
    neigh = NearestNeighbors(1)
    neigh.fit(forests_df[['longitude', 'latitude']])
    answers_train, _ = neigh.kneighbors(df_train[['longitude', 'latitude']])
    answers_test, _ = neigh.kneighbors(df_test[['longitude', 'latitude']])
    
    return answers_train, answers_test

def get_month_radius_counters_prev(df_train, df_test, fire_types, radiuses=[1.5], only_last_year=False):
    max_year = df_test.year.values.max()
    min_year = df_train.year.min()

    result_df_train = pd.DataFrame()
    result_df_test = pd.DataFrame()
    all_fire_types = []
    for radius in radiuses:
        fire_types_copy = fire_types.copy()
        for indx, val in enumerate(fire_types_copy):
            if only_last_year:
                fire_types_copy[indx] = "prev_years_similar_month_amount_last_year_" + val + "r=" + str(radius)
            else:
                fire_types_copy[indx] = "prev_years_similar_month_amount_" + val + "r=" + str(radius)
            all_fire_types.append(fire_types_copy[indx])
        result_df_train = pd.concat([result_df_train, pd.DataFrame(np.zeros((df_train.shape[0], 11)), columns=fire_types_copy)], axis=1).reset_index(drop=True)
        result_df_test = pd.concat([result_df_test, pd.DataFrame(np.zeros((df_test.shape[0], 11)), columns=fire_types_copy)], axis=1).reset_index(drop=True)

    for rad_indx, radius in tqdm(enumerate(radiuses)):
        for cur_year in range(min_year + 1, max_year + 1, 1):
            for month in range(1, 13, 1):
                for fire_indx, fire_type in enumerate(fire_types):
                    neigh_train = NearestNeighbors(10, radius=radius)
                    if only_last_year:
                        mask_train_prev = (df_train.year == cur_year - 1) & (df_train.month == month) & (df_train.fire_type_name == fire_type)
                    else:
                        mask_train_prev = (df_train.year < cur_year) & (df_train.month == month) & (df_train.fire_type_name == fire_type)
                    mask_train_cur = (df_train.year == cur_year) & (df_train.month == month)
                    if df_train[mask_train_cur].shape[0] == 0 or df_train[mask_train_prev].shape[0] == 0:
                        continue
                    neigh_train.fit(df_train[mask_train_prev][['longitude', 'latitude']])
                    answers, answers_indx = neigh_train.radius_neighbors(df_train[mask_train_cur][['longitude', 'latitude']])
                    for indx, _ in enumerate(answers):
                        answers[indx] = answers[indx].shape[0]
                    result_df_train.loc[mask_train_cur, all_fire_types[rad_indx * 11 + fire_indx]] = answers
        if only_last_year:
            result_df_train['sum_events_last_year_month_in_radius=' + str(radius)] = result_df_train.sum(axis=1)
        else:
            result_df_train['sum_events_all_years_month_in_radius=' + str(radius)] = result_df_train.sum(axis=1)
    
    for rad_indx, radius in tqdm(enumerate(radiuses)):
        for cur_year in range(min_year + 1, max_year + 1, 1):
            for month in range(1, 13, 1):
                for fire_indx, fire_type in enumerate(fire_types):
                    neigh_test = NearestNeighbors(10, radius=radius)
                    if only_last_year:
                        mask_test_prev = (df_train.year == cur_year - 1) & (df_train.month == month) & (df_train.fire_type_name == fire_type)
                    else:
                        mask_test_prev = (df_train.year < cur_year) & (df_train.month == month) & (df_train.fire_type_name == fire_type)
                    mask_test_cur = (df_test.year == cur_year) & (df_test.month == month)
                    if df_test[mask_test_cur].shape[0] == 0 or df_test[mask_test_prev].shape[0] == 0:
                        continue
                    neigh_test.fit(df_test[mask_test_prev][['longitude', 'latitude']])
                    answers, answers_indx = neigh_test.radius_neighbors(df_test[mask_test_cur][['longitude', 'latitude']])
                    for indx, _ in enumerate(answers):
                        answers[indx] = answers[indx].shape[0]
                    result_df_test.loc[mask_test_cur, all_fire_types[rad_indx * 11 + fire_indx]] = answers
        if only_last_year:
            result_df_test['sum_events_last_year_month_in_radius=' + str(radius)] = result_df_test.sum(axis=1)
        else:
            result_df_test['sum_events_all_years_month_in_radius=' + str(radius)] = result_df_test.sum(axis=1)
    
    return result_df_train, result_df_test

def evaluate(y_true, y_pred):
    gt = np.zeros_like(y_pred, dtype=np.int8)
    for index, y in enumerate(y_true):
        gt[index, y - 1] = 1
    result = {'roc_auc_micro': roc_auc_score(gt, y_pred, average='micro')}
    for ft in range(1, 12):
        gt = (y_true == ft)
        if gt.max() == gt.min():
            roc_auc = 0
        else:
            roc_auc = roc_auc_score(gt, y_pred[:, ft - 1])
        result[f'roc_auc_{ft}'] = roc_auc
    return result

Сливаем все основные фичи в один датафрейм.

In [5]:
# df_features = pd.DataFrame(columns=features_list[0].columns)
# for item in features_list:
#     df_features = pd.concat([df_features, item])
# df_features.head(3)

In [6]:
fire_types = ['технологический процесс', 'техногенный пожар', 'горение мусора', 'сжигание порубочных остатков',
'сжигание мусора', 'неконтролируемый пал', 'торфяной пожар', 'лесной пожар', 'природный пожар', 
'контролируемый пал','не подтверждено']

In [7]:
# df_features = df_features.fillna(0)

In [551]:
# df_features = pd.read_csv('basic_all_ncep_features.csv') не стоит образать внимания - тут просто были записаны все вычисления,
# чтобы каждый раз долго не экстрактить

Экстрактим самые базовые фичи. Для того, чтобы помочь катбусту понять, что 31 и 1 числа месяца близки а также 365 и 1 день в году близки будем использовать периодические функции типа cos и sin. Также покроем РФ сеткой.

In [552]:
# df_features['surface_height'] = df_features.apply(lambda x: height_surf.sel(lon=x['longitude'], lat=x['latitude'],
#                                                                             method='nearest').hgt.to_dataframe()['hgt'].values.tolist()[0], axis = 1)

# df_features['land_mask'] = df_features.apply(lambda x: land.sel(lon=x['longitude'], lat=x['latitude'],
#                                                                             method='nearest').land.to_dataframe()['land'].values.tolist()[0], axis = 1)

long_min = 10
lat_min = 40
long_step = 10
lat_step = 5

num_long_areas = (180 - long_min) // long_step + 1
num_lat_areas = (75 - lat_min) // lat_step + 1

df_features['latitude_area'] = df_features.apply(lambda x: int((x['latitude'] - lat_min) // lat_step + 1), axis=1)
df_features['longitude_area'] = df_features.apply(lambda x: int((x['longitude'] - long_min) // long_step + 1), axis=1)
df_features['area_num'] = df_features.apply(lambda x: int((x['longitude_area'] - 1) * num_lat_areas + x['latitude_area']), axis=1)
df_features['datetime'] = df_features['date'].apply(lambda x: datetime.date(int(x[:4]), int(x[5:7]), int(x[8:10])))
df_features['day_from_start']  = df_features['datetime'].apply(lambda x: (x - date(2012, 1, 1)).days)
df_features['day_of_year'] = df_features['datetime'].apply(lambda x: (x - date(x.year, 1, 1)).days)
df_features['day_of_year_cos'] = df_features['day_of_year'].apply(lambda x: np.cos(2 * np.pi * x / 365))
df_features['day_of_year_sin'] = df_features['day_of_year'].apply(lambda x: np.sin(2 * np.pi * x / 365))
df_features['day_of_month_sin'] = df_features['day'].apply(lambda x: np.sin(2 * np.pi * x / 31))
df_features['day_of_month_cos'] = df_features['day'].apply(lambda x: np.cos(2 * np.pi * x / 31))
# df_features['is_winter'] = df_features.apply(lambda x: int(x['month'] in [12, 1, 2]), axis=1)
# df_features['is_autumn'] = df_features.apply(lambda x: int(x['month'] in [9, 10, 11]), axis=1)
# df_features['is_summer'] = df_features.apply(lambda x: int(x['month'] in [6, 7, 8]), axis=1)
# df_features['is_spring'] = df_features.apply(lambda x: int(x['month'] in [3, 4, 5]), axis=1)
# df_features['year'] = df_features.apply(lambda x: int(x.date[:4]), axis=1)

In [553]:
# df_features.to_csv('basic_all_ncep_features.csv', index=False)
# df_features = pd.read_csv('basic_all_ncep_features.csv')
# df_features['date'] = df_features['date'].apply(lambda x: datetime.date(int(x[:4]), int(x[5:7]), int(x[8:10])))

Разделим выборки на тренировочную и валидационную.

In [554]:
columns_to_drop = []
n_valid_rows = 16011
df_catboost = df_features.copy()
df_catboost['date'] = df_catboost['date'].astype(str)
df_catboost = df_catboost.sort_values('date')
df_features_val = df_catboost.iloc[-n_valid_rows:].reset_index(drop=True)
df_features_train = df_catboost.iloc[: -n_valid_rows].reset_index(drop=True)

In [555]:
# df_features_train = df_features

Теперь заэкстрактим все фичи на основе KNN для тренировочной и валидационной выборок. (Отдельно их экстрактим по причине, что на тесте нам будет доступна тетовая выборка, в без тренировочной knn не обойтись, потому удобнее выччислять отдельно)

In [556]:
land_class_train, land_class_test = get_land_class_knn(df_features_train, df_features_val, merged_land_df)
nature_forest_min_dist_train, nature_forest_min_dist_test = get_forests_min_dist(df_features_train, df_features_val, forests_df)
radiuses_land = [1.0, 2.5]
min_dist_forest_train, min_dist_forest_test, am_forest_train, am_forest_test = get_radius_land_types_amount(df_features_train, df_features_val, forest_info, radiuses_land)
min_dist_field_train, min_dist_field_test, am_field_train, am_field_test = get_radius_land_types_amount(df_features_train, df_features_val, field_info, radiuses_land)
radiuses_event_month = [2.5]
events_am_rad_month_all_train, events_am_rad_month_all_test = get_month_radius_counters_prev(df_features_train, df_features_val, fire_types, radiuses_event_month)
events_am_rad_month_last_year_train, events_am_rad_month_last_year_test = get_month_radius_counters_prev(df_features_train, df_features_val, fire_types, radiuses_event_month, True)
radiuses_event = [1.0, 1.75, 2.5]
events_am_rad_train, events_am_rad_test, min_dist_event_train, min_dist_event_test, max_dist_event_train, max_dist_event_test = get_radius_counters(df_features_train, df_features_val, fire_types, radiuses_event)
radiuses_city = [5, 10, 15]
city_am_rad_train, city_am_rad_test, city_district_train, city_district_test, city_subj_train, city_subj_test, city_min_dist_train, city_min_dist_test = get_radius_cities_amount(df_features_train, df_features_val, cities_info, radiuses_city)



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.26s/it]

100%|██████████| 2/2 [00:03<00:00,  1.56s/it]


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.08s/it]

100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


0it [00:00, ?it/s]

1it [00:21, 21.46s/it]


0it [00:00, ?it/s]

1it [00:09,  9.96s/it]


0it [00:00, ?it/s]

1it [00:19, 19.34s/it]


0it [00:00, ?it/s]

1it [00:08,  8.59s/it]


  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:34<00:34, 34.04s/it]

100%|██████████| 2/2 [01:21<00:00, 40.96s/it]


0it [00:00, ?it/s]

1it [00:01,  1.17s/it]

2it [00:02,  1.16s/it]

3it [00:03,  1.22s/it]

4it [00:05,  1.28s/it]

5it [00:06,  1.29s/it]

6it [00:07,  1.35s/it]

7it [00:08,  1.26s/it]

8it [00:10,  1.29s/it]

9it [00:11,  1.35s/it]

10it [00:13,  1.39s/it]

11it [00:14,  1.35s/it]


  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:13<00:26, 13.49s/it]

 67%|██████▋   | 2/3 [00:29<00:14, 14.30s/i

На основе feature_importance были выделены фичи, которые вносят околонулевой вклад (меньше сотой доли процента и шумят в катбусте), они будут дропнуты перед началом обучения.

In [557]:
awful_features = ['humidity 100',
 'humidity_1w_mean 100',
 'humidity_2w_mean 100',
 'humidity_3w_mean 100',
 'humidity 150',
 'humidity_1w_mean 150',
 'humidity_2w_mean 150',
 'humidity_3w_mean 150',
 'trpp_1w_mean150',
 'trpp_2w_mean150',
 'trpt_1w_mean150',
 'trpt_3w_mean150',
 'humidity 200',
 'humidity_1w_mean 200',
 'humidity_2w_mean 200',
 'humidity_3w_mean 200',
 'trpp_2w_mean200',
 'trpp_3w_mean200',
 'trpt_1w_mean200',
 'trpt_3w_mean200',
 'trpp_3w_mean500',
 'trpt_1w_mean500',
 'trpt_3w_mean500',
 'trpp_1w_mean1000',
 'trpt_2w_mean1000',
 'land_mask',
 'is_winter',
 'is_summer',
 'технологический процесс_r=1.0',
 'торфяной пожар_r=1.0',
 'land_class',
 'datetime']

Соединяем всё в один большой датафрейм (очень удобно, что можно регулировать, что туда конкатить)

In [558]:
def concat_features(df_init, additional_info, col_str_list, radiuses=[], exp=False):
    if radiuses == []:
        if exp == True:
            return pd.concat([df_init, pd.DataFrame(np.exp(additional_info), columns=col_str_list)], axis=1)
        else:
            return pd.concat([df_init, pd.DataFrame(additional_info, columns=col_str_list)], axis=1)
    else:
        df_add = np.array(additional_info).copy()
        df_copy = df_init.copy()
        for indx in range(df_add.shape[0]):
            col_str_list_copy = col_str_list.copy()
            for i, col in enumerate(col_str_list_copy):
                col_str_list_copy[i] = col_str_list_copy[i] + "_r=" + str(radiuses[indx])
            df_copy = pd.concat([df_copy, pd.DataFrame(df_add[indx], columns=col_str_list_copy)], axis=1)

        return df_copy

def concat_all_features(df_init, fire_types, events_am_rad, events_am_rad_month_df, events_am_rad_month_df_last_year, 
                        radiuses_event, radiuses_land, city_am_rad, radiuses_city,
                        city_district, city_min_dist, city_subj, min_dist_forest, nature_forest_min_dist, 
                        am_forest, min_dist_field, am_field, land_class, min_dist_event, max_dist_event):

    df_init_copy = df_init.copy()

    df_init_copy = concat_features(df_init_copy, events_am_rad, fire_types, radiuses_event)
    df_init_copy = concat_features(df_init_copy, city_am_rad, ['amount_cities'], radiuses_city)
    df_init_copy = concat_features(df_init_copy, city_district, ['district'])
    df_init_copy = concat_features(df_init_copy, city_min_dist, ['cities_near_min_dist'])
    df_init_copy = concat_features(df_init_copy, city_subj, ['subject'])
    df_init_copy = concat_features(df_init_copy, min_dist_field, ['field_near_min_dist'])
    df_init_copy = concat_features(df_init_copy, min_dist_forest, ['forest_near_min_dist'])
    df_init_copy = concat_features(df_init_copy, am_field, ['amount_fields'], radiuses_land)
    df_init_copy = concat_features(df_init_copy, am_forest, ['amount_forests'], radiuses_land)
    df_init_copy = concat_features(df_init_copy, land_class, ['land_class'])
    df_init_copy = concat_features(df_init_copy, nature_forest_min_dist, ['nature_forest_min_dist'])

    fire_types_copy = fire_types.copy()
    for indx, val in enumerate(fire_types_copy):
        fire_types_copy[indx] = "max_dist_" + val
    
    df_init_copy = concat_features(df_init_copy, max_dist_event, fire_types_copy)

    fire_types_copy = fire_types.copy()
    for indx, val in enumerate(fire_types_copy):
        fire_types_copy[indx] = "min_dist_" + val

    df_init_copy = concat_features(df_init_copy, min_dist_event, fire_types_copy)
    
    df_init_copy = pd.concat([df_init_copy, events_am_rad_month_df], axis=1)
#     df_init_copy = pd.concat([df_init_copy, events_am_rad_month_df_last_year], axis=1)
    
    print(df_init_copy.shape)

    return df_init_copy


df_features_with_counters_train = concat_all_features(df_features_train, fire_types,
                                                      events_am_rad_train, events_am_rad_month_all_train, events_am_rad_month_last_year_train, 
                                                      radiuses_event, radiuses_land, city_am_rad_train, radiuses_city, city_district_train,
                                                      city_min_dist_train, city_subj_train,
                                                      min_dist_forest_train, nature_forest_min_dist_train, am_forest_train,
                                                      min_dist_field_train, am_field_train,
                                                      land_class_train, min_dist_event_train, max_dist_event_train)


df_features_with_counters_test = concat_all_features(df_features_val, fire_types,
                                                     events_am_rad_test, events_am_rad_month_all_test, events_am_rad_month_last_year_test, 
                                                     radiuses_event, radiuses_land,city_am_rad_test, radiuses_city, city_district_test,
                                                     city_min_dist_test, city_subj_test,
                                                     min_dist_forest_test, nature_forest_min_dist_test, am_forest_test,
                                                     min_dist_field_test, am_field_test,
                                                     land_class_test, min_dist_event_test, max_dist_event_test)

# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month(df_features_with_counters_train, df_features_with_counters_test, 'area_num')
# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month(df_features_with_counters_train, df_features_with_counters_test, 'subject')
# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month(df_features_with_counters_train, df_features_with_counters_test, 'district')
# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month_only_prev(df_features_with_counters_train, df_features_with_counters_test, 'district')
# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month_only_prev(df_features_with_counters_train, df_features_with_counters_test, 'subject')
# df_features_with_counters_train, df_features_with_counters_test = get_count_subj_month_only_prev(df_features_with_counters_train, df_features_with_counters_test, 'area_num')

(158860, 241)
(16011, 241)


Простой чекпоинт для сохранения промежуточных результатов

In [559]:
# df_features_with_counters_train.to_csv('df_features_with_counters_train.csv', index=False)
# df_features_with_counters_test.to_csv('df_features_with_counters_test.csv', index=False)
# df_features_with_counters_train = pd.read_csv('df_features_with_counters_train.csv')
# df_features_with_counters_test = pd.read_csv('df_features_with_counters_test.csv')

Чистим данные для тренировки и валидации.

In [560]:
drop_features=['fire_type', 'date', 'fire_type_name', 'latitude_area',
               'longitude_area'] + awful_features
cols = list(df_features_with_counters_test.drop(drop_features, axis=1).columns)

desired_cat_features = ['month', 'district', 'subject', 'weekday']
cat_features = []
for feature in desired_cat_features:
    cat_features.append(cols.index(feature))
    
X_train = df_features_with_counters_train.drop(drop_features, axis=1)
Y_train = df_features_with_counters_train.fire_type
X_val = df_features_with_counters_test.drop(drop_features, axis=1)
Y_val = df_features_with_counters_test.fire_type

true_val = df_features_with_counters_test.fire_type.tolist()
X_train_all = pd.concat([X_train, X_val]).reset_index(drop=True)
Y_train_all = pd.concat([Y_train, Y_val]).reset_index(drop=True)

В данной закомменченной части кода был опробован стакинг моделей (мб у кого-то он лучше зажжёт)

In [561]:
# from sklearn import model_selection
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB 
# from sklearn.ensemble import RandomForestClassifier
# from mlxtend.classifier import StackingCVClassifier

# n_jobs = -1

# clf1 = RandomForestClassifier(n_estimators=100, verbose=True, n_jobs=n_jobs)
# clf2 = KNeighborsClassifier()

# lr = catboost.CatBoostClassifier(loss_function='MultiClass',
#                                   verbose=1000, iterations=1500, task_type='CPU', cat_features=cat_features)

# sclf = StackingCVClassifier(classifiers=[clf1, clf2], 
#                           meta_classifier=lr, use_features_in_secondary=True, verbose=1, n_jobs=n_jobs, cv=50)

# sclf.fit(X_train, Y_train)

In [562]:
# import pickle
# drop_susp_feat = ['events_amount_passed_in_area_num', 'events_amount_passed_in_subject', 
#                   'events_amount_passed_in_district']
# drop_susp_feat = []
# df_features_with_counters_train.drop(columns=drop_susp_feat).to_csv('./solution/df_features_train.csv',
#                                                                     index=False)
# 
# with open('solution/model.pickle', 'wb') as fout:
#     pickle.dump(sclf, fout, protocol=pickle.HIGHEST_PROTOCOL)

Тренируем катбуст и дополнительно выводим трениовочные и валидационные метрики + confusion matrix. Анализируем результаты, докидываем идеи...

In [566]:
print("Start training!") 

clf = catboost.CatBoostClassifier(loss_function='MultiClass',
                                  verbose=10, iterations=10000, 
                                  od_type = "Iter", od_wait = 200, cat_features=cat_features,
                                  eval_metric='MultiClass', max_depth=6)
clf.fit(X_train, Y_train, eval_set=(X_val, Y_val))
pred_train = clf.predict_proba(X_train)
pred_val = clf.predict_proba(X_val)

class_val = clf.predict(X_val)
train_scores = evaluate(Y_train, pred_train)
val_scores = evaluate(Y_val, pred_val)
print("Train scores:")
for k, v in train_scores.items():
    print("%s\t%f" % (k, v))
print("Validation scores:")
for k, v in val_scores.items():
    print("%s\t%f" % (k, v))
conf_matrix = confusion_matrix(true_val, class_val, labels=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
conf_matrix

Start training!
0:	learn: 2.3161215	test: 2.3172767	best: 2.3172767 (0)	total: 651ms	remaining: 1h 48m 27s
10:	learn: 1.8578534	test: 1.8668354	best: 1.8668354 (10)	total: 6.19s	remaining: 1h 33m 40s
20:	learn: 1.6371092	test: 1.6488593	best: 1.6488593 (20)	total: 11.8s	remaining: 1h 33m 10s
30:	learn: 1.5013154	test: 1.5200859	best: 1.5200859 (30)	total: 17.2s	remaining: 1h 32m 24s
40:	learn: 1.4074460	test: 1.4403943	best: 1.4403943 (40)	total: 22.7s	remaining: 1h 31m 57s
50:	learn: 1.3404872	test: 1.3848741	best: 1.3848741 (50)	total: 28.2s	remaining: 1h 31m 37s
60:	learn: 1.2902764	test: 1.3459033	best: 1.3459033 (60)	total: 33.6s	remaining: 1h 31m 12s
70:	learn: 1.2521702	test: 1.3192719	best: 1.3192719 (70)	total: 38.9s	remaining: 1h 30m 45s
80:	learn: 1.2224874	test: 1.2989529	best: 1.2989529 (80)	total: 44.3s	remaining: 1h 30m 29s
90:	learn: 1.1969524	test: 1.2853104	best: 1.2853104 (90)	total: 49.9s	remaining: 1h 30m 28s
100:	learn: 1.1768003	test: 1.2741444	best: 1.2741444 (1

860:	learn: 0.9010286	test: 1.2015726	best: 1.2013292 (851)	total: 7m 40s	remaining: 1h 21m 22s
870:	learn: 0.8997553	test: 1.2016264	best: 1.2013292 (851)	total: 7m 45s	remaining: 1h 21m 15s
880:	learn: 0.8987355	test: 1.2011874	best: 1.2011874 (880)	total: 7m 50s	remaining: 1h 21m 9s
890:	learn: 0.8976672	test: 1.2000505	best: 1.2000505 (890)	total: 7m 55s	remaining: 1h 21m 4s
900:	learn: 0.8964863	test: 1.2000171	best: 1.2000171 (900)	total: 8m 1s	remaining: 1h 20m 58s
910:	learn: 0.8952676	test: 1.2000835	best: 1.1999846 (902)	total: 8m 6s	remaining: 1h 20m 53s
920:	learn: 0.8940597	test: 1.1992095	best: 1.1991205 (919)	total: 8m 11s	remaining: 1h 20m 47s
930:	learn: 0.8928172	test: 1.1993329	best: 1.1990655 (923)	total: 8m 16s	remaining: 1h 20m 40s
940:	learn: 0.8918616	test: 1.1981984	best: 1.1981546 (939)	total: 8m 22s	remaining: 1h 20m 34s
950:	learn: 0.8905365	test: 1.1982456	best: 1.1981031 (943)	total: 8m 27s	remaining: 1h 20m 29s
960:	learn: 0.8894015	test: 1.1980255	best: 

1700:	learn: 0.8267926	test: 1.1920157	best: 1.1918427 (1685)	total: 15m 9s	remaining: 1h 13m 58s
1710:	learn: 0.8261523	test: 1.1925077	best: 1.1918427 (1685)	total: 15m 15s	remaining: 1h 13m 53s
1720:	learn: 0.8254644	test: 1.1924408	best: 1.1918427 (1685)	total: 15m 20s	remaining: 1h 13m 47s
1730:	learn: 0.8248064	test: 1.1925076	best: 1.1918427 (1685)	total: 15m 25s	remaining: 1h 13m 42s
1740:	learn: 0.8242338	test: 1.1925400	best: 1.1918427 (1685)	total: 15m 31s	remaining: 1h 13m 37s
1750:	learn: 0.8236280	test: 1.1924511	best: 1.1918427 (1685)	total: 15m 36s	remaining: 1h 13m 32s
1760:	learn: 0.8230084	test: 1.1925447	best: 1.1918427 (1685)	total: 15m 42s	remaining: 1h 13m 27s
1770:	learn: 0.8225049	test: 1.1924817	best: 1.1918427 (1685)	total: 15m 47s	remaining: 1h 13m 21s
1780:	learn: 0.8217834	test: 1.1925272	best: 1.1918427 (1685)	total: 15m 52s	remaining: 1h 13m 16s
1790:	learn: 0.8211668	test: 1.1925949	best: 1.1918427 (1685)	total: 15m 57s	remaining: 1h 13m 10s
1800:	learn

array([[  33,    0,    0,    4,    0,    0,    0,    0,    3,   10,    4],
       [   0,    0,    1,    5,    2,    9,    0,    1,   17,   21,   10],
       [   1,    0,   34,   16,    0,   10,    0,    0,   93,   14,   73],
       [   4,    0,   18,  282,    4,   18,    0,    1,   45,  223,   14],
       [   0,    0,   16,   38,    7,   36,    0,    2,   18,   71,   13],
       [   0,    0,    0,   45,    2, 2869,    0,   15,  617,  524,  166],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    1,    4,    0,  118,    0,   29,   82,  102,    9],
       [   8,    0,   15,   26,    1,  571,    0,    7, 2884,  422,  917],
       [   8,    0,    8,  207,    1,  505,    0,    4,  312, 1528,  179],
       [   5,    0,   10,   29,    2,  268,    0,    3, 1004,   90, 1243]])

Посмотрим на важность фичей:

In [567]:
feature_scores = {}
for score, name in zip(clf.feature_importances_, clf.feature_names_):
    feature_scores[name] = score
    
feature_scores = sorted(feature_scores.items(), key=lambda kv: kv[1])
feature_scores

[('prev_years_similar_month_amount_торфяной пожарr=2.5', 0.010508118812826314),
 ('srfpt_1w_mean100', 0.017966383501068238),
 ('is_autumn', 0.021547257801423882),
 ('prev_years_similar_month_amount_техногенный пожарr=2.5',
  0.024997557488270452),
 ('trpt_2w_mean500', 0.026731834280032556),
 ('trpt_2w_mean200', 0.037748477559602936),
 ('srfpt_2w_mean500', 0.04355880435948699),
 ('trpp_1w_mean200', 0.04859835549966615),
 ('prev_years_similar_month_amount_технологический процессr=2.5',
  0.04880547468168361),
 ('srfpt_2w_mean1000', 0.05647599475552961),
 ('trpt_1w_mean1000', 0.056936502705484476),
 ('srfpt_1w_mean1000', 0.057282644257477776),
 ('height_1w_mean200', 0.06291225980421451),
 ('trpt_2w_mean150', 0.06541959543888888),
 ('торфяной пожар_r=2.5', 0.07225413415668692),
 ('trpp_2w_mean1000', 0.07636251549117543),
 ('srfpt_2w_mean150', 0.07636865312866442),
 ('trpp_1w_mean500', 0.07779009366471516),
 ('trpp_3w_mean1000', 0.07834211400243718),
 ('height_2w_mean150', 0.079663834840755

In [568]:
accuracy_score(class_val.reshape(-1), Y_val) #0.688 0.5707950783836113

0.5564299544063457

In [ ]:
drop_susp_feat = ['events_amount_passed_last_month_in_area_num', 'events_amount_passed_last_month_in_subject', 
                  'events_amount_passed_last_month_in_district']
# drop_susp_feat = []
df_features_with_counters_train.drop(columns=drop_susp_feat).to_csv('./solution/df_features_train.csv', index=False)

по городу (5, 10) и до 3 радиус и ближ расст до города и область - кат фича месяц, область
AUC abs max = 0.9272, loss abs min 1.1857

In [ ]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(clf, fout, protocol=pickle.HIGHEST_PROTOCOL)